In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nf-uq-nids-v2-sample/NF-UQ-NIDS-V2-Sample.csv
/kaggle/input/nf-uq-nids-v2-sample/NF-UQ-NIDS-V2_Sample-2.csv


In [4]:
!rm -rf /kaggle/working/*

In [5]:
df = pd.read_csv('/kaggle/input/nf-uq-nids-v2-sample/NF-UQ-NIDS-V2_Sample-2.csv')

In [6]:
np.unique(df['Attack'])

array(['Analysis', 'Backdoor', 'Benign', 'Bot', 'Brute Force', 'DDoS',
       'DoS', 'Exploits', 'Fuzzers', 'Generic', 'Infilteration',
       'Reconnaissance', 'Shellcode', 'Theft', 'Worms', 'injection',
       'mitm', 'password', 'ransomware', 'scanning', 'xss'], dtype=object)

In [7]:
from sklearn.preprocessing import LabelEncoder
LE1 = LabelEncoder()

In [8]:
def processX(X):
    #ip addresses are in format xxx.xxx.xxx.xxx which is not compatible with the models
    srcaddr= LE1.fit_transform(X['IPV4_SRC_ADDR'])
    X['IPV4_SRC_ADDR'] = pd.Series(srcaddr)
    dstaddr= LE1.fit_transform(X['IPV4_DST_ADDR'])
    X['IPV4_DST_ADDR'] = pd.Series(dstaddr)
    

    from sklearn.impute import SimpleImputer 
    imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
    
    X_object = X.select_dtypes("object")
    X_int = X.select_dtypes(exclude = "object")
    X_columns = X_int.columns #this stores column names 

    imputer.fit(X_int)
    X_int = imputer.transform(X_int)
    X_int = pd.DataFrame(X_int)
    
    X_int.columns = X_columns #corrects column name
    
    X_object = X_object.fillna("unknown")
    X = X_object.join(X_int)
    from sklearn.preprocessing import StandardScaler, MinMaxScaler
    scaler = MinMaxScaler()
    # store the columns
    cols = X.columns
    
    # transform the data
    X_scaled = scaler.fit_transform(X)
    
    X_scaled = pd.DataFrame(X_scaled, columns = cols)
     
    
    return X_scaled

In [9]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

In [10]:
X=df.drop(['Attack','Dataset','Label','Attack_Class'],axis=1)
y=df['Attack']
X=processX(X)

In [11]:

# Assuming 'y_train' and 'y_test' are your original target labels
target_classes = ['Benign', 'DDoS', 'DoS', 'Brute Force', ]#'injection'] 

# Filter the data
X = X[np.isin(y, target_classes)]
y = y[np.isin(y, target_classes)]


In [12]:
multilbl = LE.fit_transform(y)
y = pd.Series(multilbl)
# Access the original classes
classes = LE.classes_
# Print the mapping
for i, cls in enumerate(classes):
    print(f"{i}: {cls}")

0: Benign
1: Brute Force
2: DDoS
3: DoS


In [13]:
print(type(classes))

<class 'numpy.ndarray'>


In [14]:
from tensorflow.keras.utils import to_categorical
y_cat = to_categorical(y)

In [15]:
from sklearn.model_selection import train_test_split
x_train_dl, x_test_dl, y_train_dl, y_test_dl = train_test_split(X, y, test_size=0.2,random_state=55)
x_train_dl, x_val_dl, y_train_dl, y_val_dl = train_test_split(x_train_dl, y_train_dl, test_size=0.25,random_state=55)

In [16]:
input_feature=x_train_dl.shape[1]
output_classes=1#binary classification

In [17]:
import tensorflow as tf
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout, BatchNormalization, Reshape, SimpleRNN, Flatten

In [18]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0005,
    decay_steps=10000,
    decay_rate=0.96,
    staircase=True
)

In [19]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [20]:
def CNN_Model(input_feature,output_classes):
    model = Sequential([
        Input(shape=(input_feature,1)),
        Dense(128, activation='relu'),
        Dense(64, activation = 'relu'),
        Dense(32, activation = 'relu'),
        Dense(16, activation = 'relu'),
        Flatten(),
        Dense(output_classes, activation="sigmoid")
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [21]:

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
# model_cnn = CNN_Model()
# model_cnn.summary()

In [22]:
# history_cnn = model_cnn.fit(x_train_dl ,y_train_dl, verbose=1, epochs=100, batch_size=256,validation_data=(x_val_dl,y_val_dl),callbacks=[early_stopping])

In [23]:

# Assuming you've identified unique attack types
unique_attacks = sorted(list(set(y)))

In [24]:
num_classes = len(target_classes)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [26]:
def train_random_forest(X, y):
    rf = RandomForestClassifier()
    rf.fit(X, y)
    return rf

In [27]:
x_train_dl

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE
2829481,0.538815,0.393515,0.274445,0.001221,0.023529,0.028226,2.466209e-06,0.000003,0.000000e+00,0.000000,...,0.00000,0.0,0.007813,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1360815,0.538838,0.518914,0.274445,0.001221,0.023529,0.028226,2.466209e-06,0.000003,0.000000e+00,0.000000,...,0.00000,0.0,0.007813,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1566383,0.538792,0.685313,0.274746,0.001221,0.023529,0.028226,1.215233e-06,0.000000,2.050191e-07,0.000008,...,0.00000,0.0,0.007813,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
505344,0.538815,0.452522,0.274595,0.001221,0.023529,0.028226,2.466209e-06,0.000003,0.000000e+00,0.000000,...,0.00000,0.0,0.007813,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3310944,0.538792,0.016663,0.274821,0.001221,0.023529,0.028226,2.466209e-06,0.000003,0.000000e+00,0.000000,...,0.00000,0.0,0.007813,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2276609,0.538815,0.806424,0.274445,0.001221,0.066667,0.758065,4.646480e-07,0.000003,0.000000e+00,0.000000,...,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3343964,0.347871,0.846952,0.234099,0.001221,0.023529,0.028226,4.682222e-06,0.000011,5.878924e-06,0.000038,...,0.00003,0.0,1.000000,0.410208,0.0,0.0,0.0,0.0,0.0,0.0
2814049,0.538518,0.524117,0.272036,0.096834,0.023529,0.000000,3.574216e-07,0.000000,2.050191e-07,0.000008,...,0.00000,0.0,0.015625,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
175869,0.538792,0.898970,0.274595,0.001221,0.066667,0.758065,4.646480e-07,0.000003,0.000000e+00,0.000000,...,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

In [29]:
selected_features={
    'Benign': ['IPV4_SRC_ADDR','IPV4_DST_ADDR','L4_DST_PORT','L7_PROTO','OUT_BYTES','OUT_PKTS','FLOW_DURATION_MILLISECONDS','DURATION_IN','MIN_TTL','MAX_TTL','LONGEST_FLOW_PKT','MIN_IP_PKT_LEN','DST_TO_SRC_AVG_THROUGHPUT','TCP_WIN_MAX_IN','TCP_WIN_MAX_OUT'],
    'DDoS': ['IPV4_SRC_ADDR','L4_DST_PORT','L7_PROTO','IN_PKTS','OUT_BYTES','OUT_PKTS','DURATION_OUT','MIN_TTL','MAX_TTL','LONGEST_FLOW_PKT','MAX_IP_PKT_LEN','SRC_TO_DST_AVG_THROUGHPUT','DST_TO_SRC_AVG_THROUGHPUT','NUM_PKTS_UP_TO_128_BYTES','NUM_PKTS_128_TO_256_BYTES'],
    'DoS': ['IPV4_DST_ADDR','L4_DST_PORT','PROTOCOL','L7_PROTO','TCP_FLAGS','FLOW_DURATION_MILLISECONDS','DURATION_OUT','LONGEST_FLOW_PKT','SHORTEST_FLOW_PKT','MAX_IP_PKT_LEN','SRC_TO_DST_AVG_THROUGHPUT', 'NUM_PKTS_UP_TO_128_BYTES','NUM_PKTS_128_TO_256_BYTES','TCP_WIN_MAX_IN','TCP_WIN_MAX_OUT'],
    'Brute Force': ['IPV4_SRC_ADDR','IPV4_DST_ADDR','OUT_BYTES','OUT_PKTS','CLIENT_TCP_FLAGS','FLOW_DURATION_MILLISECONDS','DURATION_IN','LONGEST_FLOW_PKT','SHORTEST_FLOW_PKT','MAX_IP_PKT_LEN','SRC_TO_DST_AVG_THROUGHPUT','DST_TO_SRC_AVG_THROUGHPUT','NUM_PKTS_UP_TO_128_BYTES','NUM_PKTS_128_TO_256_BYTES','TCP_WIN_MAX_IN'],
    #'injection': ['IPV4_SRC_ADDR', 'IPV4_DST_ADDR', 'L4_DST_PORT', 'TCP_FLAGS', 'CLIENT_TCP_FLAGS', 'MAX_TTL', 'LONGEST_FLOW_PKT', 'MIN_IP_PKT_LEN', 'RETRANSMITTED_OUT_BYTES', 'RETRANSMITTED_OUT_PKTS', 'SRC_TO_DST_AVG_THROUGHPUT', 'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT', 'ICMP_TYPE', 'DNS_QUERY_ID']
}

In [30]:
selected_features[classes[1]]

['IPV4_SRC_ADDR',
 'IPV4_DST_ADDR',
 'OUT_BYTES',
 'OUT_PKTS',
 'CLIENT_TCP_FLAGS',
 'FLOW_DURATION_MILLISECONDS',
 'DURATION_IN',
 'LONGEST_FLOW_PKT',
 'SHORTEST_FLOW_PKT',
 'MAX_IP_PKT_LEN',
 'SRC_TO_DST_AVG_THROUGHPUT',
 'DST_TO_SRC_AVG_THROUGHPUT',
 'NUM_PKTS_UP_TO_128_BYTES',
 'NUM_PKTS_128_TO_256_BYTES',
 'TCP_WIN_MAX_IN']

In [31]:
models={}

In [32]:
# Train and evaluate for each attack type (OvR)
for i in range(0,4):#, [model,attack] in enumerate(models):

    attack=classes[i]
    # Create OvR target variable
    y_train_ovr = (y_train_dl == i).astype(int)
    y_test_ovr = (y_test_dl == i).astype(int)
    y_val_ovr = (y_val_dl == i).astype(int)
    

    print(f"Training for {attack}")
    
    print("Selecting features")
    X_train_selected=x_train_dl[selected_features[attack]]
    X_test_selected=x_test_dl[selected_features[attack]]
    X_val_selected=x_val_dl[selected_features[attack]]
    
    
    x_train_dl_rs = X_train_selected.values.reshape(X_train_selected.shape[0], X_train_selected.shape[1],1)
    x_val_dl_rs = X_val_selected.values.reshape(X_val_selected.shape[0], X_val_selected.shape[1],1)
    x_test_dl_rs = X_test_selected.values.reshape(X_test_selected.shape[0], X_test_selected.shape[1],1)
    
    print("Training CNN model")
    input_feature=X_train_selected.shape[1]
    output_classes=1#binary classification
    model = CNN_Model(input_feature,output_classes)
    model.fit(x_train_dl_rs, y_train_ovr, epochs=100, batch_size=256,validation_data=(x_val_dl_rs,y_val_ovr),callbacks=[early_stopping], verbose=1)  # Adjust epochs and batch size

    # Evaluate the model
    print("Evaluating Model")
    loss, accuracy = model.evaluate(x_test_dl_rs, y_test_ovr, verbose=1)
    print(f"Accuracy for {attack}: {accuracy}")
    models[attack]=model
    
    print("Saving Model")
    model.save(f"model_{attack}.h5")
    print("Model Saved!")


Training for Benign
Selecting features
Training CNN model
Epoch 1/100


I0000 00:00:1732646600.936840     111 service.cc:145] XLA service 0x7e1ba8004180 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732646600.936884     111 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  88/7608 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.6227 - loss: 0.6281 

I0000 00:00:1732646602.443052     111 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


7608/7608 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9626 - loss: 0.0866 - val_accuracy: 0.9939 - val_loss: 0.0216
Epoch 2/100
7608/7608 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.9934 - loss: 0.0215 - val_accuracy: 0.9943 - val_loss: 0.0186
Epoch 3/100
7608/7608 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.9941 - loss: 0.0196 - val_accuracy: 0.9943 - val_loss: 0.0187
Epoch 4/100
7608/7608 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9946 - loss: 0.0182 - val_accuracy: 0.9953 - val_loss: 0.0161
Epoch 5/100
7608/7608 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9949 - loss: 0.0168 - val_accuracy: 0.9954 - val_loss: 0.0167
Epoch 6/100
7608/7608 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.9949 - loss: 0.0167 - val_accuracy: 0.9958 - val_loss: 0.0150
Epoch 7/100
7608/7608 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9953 - loss: 0.0158 - val_accuracy: 0.9952 - val_loss: 0.0152
Epoch 8/100
7608/7608 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.9953 - loss: 0.015

In [33]:
import tensorflow as tf

# Function to load a model from an h5 file
def load_model_from_h5(file_path):
    model = tf.keras.models.load_model(file_path)
    return model

# Load all models
models = [load_model_from_h5(f"/kaggle/working/model_{attack}.h5") for attack in classes]

# Function to make a prediction using the OvR models
def predict_class(x):
    predictions = []
    i=0
    for model in models:
        x_sel=x[selected_features[classes[i]]]
        prediction = model.predict(x_sel,verbose=0)
        predictions.append(prediction)
        i=i+1

    # Convert predictions to class probabilities
    probabilities = np.array(predictions)
    
    # Predict the class with the highest probability
    predicted_classes = np.argmax(probabilities, axis=0)

    return predicted_classes


In [34]:
predicted_class = predict_class(x_test_dl)
print("Predicted class:", predicted_class)

Predicted class: [[3]
 [3]
 [0]
 ...
 [2]
 [0]
 [3]]


In [35]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
# Evaluate the model
accuracy = accuracy_score(y_test_dl, predicted_class)
precision = precision_score(y_test_dl, predicted_class, average='weighted')
recall = recall_score(y_test_dl, predicted_class, average='weighted')
f1 = f1_score(y_test_dl, predicted_class, average='weighted')

confusion_mat = confusion_matrix(y_test_dl, predicted_class)
confusion_rep = classification_report(y_test_dl, predicted_class)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("Confusion Matrix:\n", confusion_mat)

Accuracy: 0.9924207179367099
Precision: 0.9924496497900511
Recall: 0.9924207179367099
F1-Score: 0.9924241747476793
Confusion Matrix:
 [[250661      0    146    444]
 [     2   1232     36      3]
 [   324      0 215009   2610]
 [   178      0   1177 177316]]


In [36]:
print(confusion_rep)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    251251
           1       1.00      0.97      0.98      1273
           2       0.99      0.99      0.99    217943
           3       0.98      0.99      0.99    178671

    accuracy                           0.99    649138
   macro avg       0.99      0.99      0.99    649138
weighted avg       0.99      0.99      0.99    649138

